# Recommender System

Recommender systems are no joke. They have found enterprise application a long time ago by helping all the top players in the online market place. Amazon, Netflix, Google and many others have been using the technology to curate content and products for its customers. Amazon recommends products based on your purchase history, user ratings of the product etc. Netflix recommends movies and TV shows all made possible by highly efficient recommender systems.

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. In this notebook, I will implement Collaborative and Content-based recommendation systems using Python and the Pandas library.


# Collaborative Filtering

Collaborative Filtering is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one I will be using here is going to be based on the Pearson Correlation Function.

## Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")


## Advantages and Disadvantages of Collaborative Filtering

### Advantages
- Takes other user's ratings into consideration
- Doesn't need to study or extract information from the recommended item
- Adapts to the user's interests which might change over time

### Disadvantages
- Approximation function can be slow
- There might be a low of amount of users to approximate
- Privacy issues when trying to learn the user's preferences

## Dataset  : MovieLens

GroupLens Research has collected and made available rating data sets from the MovieLens web site (https://movielens.org). The data sets were collected over various periods of time, depending on the size of the set. Before using these data sets, please review their README files for the usage licenses and other details.

Dataset Contain

MovieLens 25M movie ratings. Stable benchmark dataset. 25 million ratings and one million tag applications applied to 62,000 movies by 162,000 users. Includes tag genome data with 15 million relevance scores across 1,129 tags. Released 12/2019

**Link : https://grouplens.org/datasets/movielens/**

# Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd

# <h1><center>Collaborative Filtering</center></h1>

After preprocessing, the process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

**==>** The data is distributed in four different CSV files which are named as ratings, movies, links and tags. For building this recommender we will only consider the ratings and the movies datasets.

**==>** The ratings dataset consists of number of observations and each observation is a record of the ID for the user who rated the movie (userId), the ID of the Movie that is rated (movieId), the rating given by the user for that particular movie (rating) and the time at which the rating was recorded(timestamp).

**==>** The movies dataset consists of the ID of the movies(movieId), the corresponding title (title) and genre of each movie(genres).

## Loading Data

Movies data file read

In [2]:
movies_df = pd.read_csv('ml-25m/movies.csv')
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [3]:
# shape of Movie dataframe
movies_df.shape

(62423, 3)

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


Rating file read

In [5]:
ratings_df = pd.read_csv('ml-25m/ratings.csv')
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [6]:
# shape of rating dataframe
ratings_df.shape

(25000095, 4)

In [7]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


# Preprocessing

## Check null value

In [8]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [9]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

## Preprocessing movies dataframe

In [10]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. 

Let's remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

Using regular expressions to find a year stored between parentheses<br>
We specify the parantheses so we don't conflict with movies that have years in their titles

In [11]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year']

0        (1995)
1        (1995)
2        (1995)
3        (1995)
4        (1995)
          ...  
62418    (2018)
62419    (2001)
62420    (2018)
62421    (2001)
62422    (1962)
Name: year, Length: 62423, dtype: object

In [12]:
#Removing paranthesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
62418    2018
62419    2001
62420    2018
62421    2001
62422    1962
Name: year, Length: 62423, dtype: object

In [13]:
movies_df['title']

0                          Toy Story (1995)
1                            Jumanji (1995)
2                   Grumpier Old Men (1995)
3                  Waiting to Exhale (1995)
4        Father of the Bride Part II (1995)
                        ...                
62418                             We (2018)
62419             Window of the Soul (2001)
62420                      Bad Poems (2018)
62421                   A Girl Thing (2001)
62422        Women of Devil's Island (1962)
Name: title, Length: 62423, dtype: object

In [14]:
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title']

<ipython-input-14-3b9391ae2061>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


0                          Toy Story 
1                            Jumanji 
2                   Grumpier Old Men 
3                  Waiting to Exhale 
4        Father of the Bride Part II 
                     ...             
62418                             We 
62419             Window of the Soul 
62420                      Bad Poems 
62421                   A Girl Thing 
62422        Women of Devil's Island 
Name: title, Length: 62423, dtype: object

In [15]:
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [16]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


### genres feature drop

In [17]:
movies_df = movies_df.drop('genres', 1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


## Preprocessing ratings dataframe

In [18]:
# Rating data
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


### timestamp feature drop

In [19]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


# creating an input user to recommend movies to

**=> To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .**

In [20]:
userInput = [
            {'title':'Toy Story', 'rating':5},
            {'title':'Jumanji-4', 'rating':3.5},
            {'title':'Hero', 'rating':2},
            {'title':'Grumpier Old Men', 'rating':5},
            {'title':'Jungle Book', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Toy Story,5.0
1,Jumanji-4,3.5
2,Hero,2.0
3,Grumpier Old Men,5.0
4,Jungle Book,4.5


Add movieId to input user

In [21]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

inputMovies

,movieId,title,rating
0,1,Toy Story,5.0
1,3,Grumpier Old Men,5.0
2,2252,Hero,2.0
3,146138,Hero,2.0
4,6743,Jungle Book,4.5


## The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [22]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
70,2,1,3.5
254,3,1,4.0
910,4,1,3.0
1152,5,1,4.0
1304,8,1,4.0


We now group up the rows by user ID.

In [23]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

E.g., UserID = 296

lets look at one of the usersID, MovileID, rating

In [24]:
userSubsetGroup.get_group(296)

,userId,movieId,rating
37776,296,1,3.5


In [25]:
#user with id 1130 having all 5 similar movies watched
len(userSubsetGroup.get_group(296))

1

E.g., UserID = 110

lets look at one of the usersID, MovileID, rating

In [26]:
userSubsetGroup.get_group(110)

,userId,movieId,rating
14294,110,1,4.0


In [27]:
#user with id 110 having only 1 similar movie watched
len(userSubsetGroup.get_group(110))

1

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [28]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [29]:
#Top most user with id 75 having all 5 similar moves watched
userSubsetGroup[0]

(2177,
         userId  movieId  rating
 314082    2177        1     4.0
 314084    2177        3     4.0
 315170    2177     2252     4.0
 316992    2177     6743     3.5)

In [30]:
#name of top user group
userSubsetGroup[0][0]

2177

In [31]:
#dataframe of top user group
userSubsetGroup[0][1]

,userId,movieId,rating
314082,2177,1,4.0
314084,2177,3,4.0
315170,2177,2252,4.0
316992,2177,6743,3.5


# Similarity of users to input user

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [32]:
userSubsetGroup = userSubsetGroup[0:100]
userSubsetGroup[0]

(2177,
         userId  movieId  rating
 314082    2177        1     4.0
 314084    2177        3     4.0
 315170    2177     2252     4.0
 316992    2177     6743     3.5)

In [33]:
userSubsetGroup[44]

(3360,
         userId  movieId  rating
 488968    3360        1     4.0
 488969    3360        3     4.0
 489309    3360     2252     3.0)

In [34]:
userSubsetGroup[99]

(20471,
          userId  movieId  rating
 3106708   20471        1     4.0
 3106710   20471        3     2.5
 3107096   20471     2252     4.0)

**=>> Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient**

In [35]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    #Get the N (total similar movies watched) for the formula 
    nRatings = len(group)
    
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    
    #Now let's calculate the pearson correlation between two users, so called, x and y

    #scipy.stats import pearsonr
    #pearsonr(tempRatingList,tempGroupList)[0]

    #For hard code based
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/np.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [36]:
pearsonCorrelationDict.items()

dict_items([(2177, -0.17407765595569785), (7288, -0.2345088235604828), (8619, 0.502518907629606), (9448, 0.17407765595569785), (12719, 0.5222329678670935), (13990, -0.3419834193661059), (15598, -0.22473328748774737), (17941, 0.502518907629606), (18306, 0.07036672182012485), (18560, 0.13937366833451512), (20055, 0.6360199717944434), (24227, -0.5151515151515151), (36618, 0.8181818181818182), (47371, -0.4923659639173309), (48499, 0.4923659639173309), (60861, -0.050964719143762556), (61010, -0.16140004156875074), (76662, 0.28880007514798783), (80974, 0.4061811972299616), (84166, 0.055989251095585425), (84658, 0.41770635491817765), (92046, 0.07036672182012485), (92854, 0.48029240224721587), (97452, 0.25362863675089403), (105737, 0), (109090, 0.9393939393939394), (109731, 0.3939393939393939), (131506, 0.8181818181818182), (135429, 0.502518907629606), (138453, 0.19728801008312422), (152107, 0.4130679975805081), (160922, 0.5222329678670935), (12, 0.4999999999999999), (166, 1.0000000000000018),

### Pearson Correlation of each MovieId

In [37]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.head()

,0
2177,-0.174078
7288,-0.234509
8619,0.502519
9448,0.174078
12719,0.522233


## Similarity

In [38]:
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.174078,2177
1,-0.234509,7288
2,0.502519,8619
3,0.174078,9448
4,0.522233,12719


### The top x similar users to input user

**Desending Oder of similarityIndex**

Now let's get the top 1000 users that are most similar to the input.

In [39]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
97,1.0,19835
33,1.0,166
44,1.0,3360
96,1.0,19101
46,1.0,4388


### Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearson Data Frame** from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [40]:
topUsersRating = topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,19835,1,3.5
1,1.0,19835,2,3.5
2,1.0,19835,3,3.5
3,1.0,19835,5,2.5
4,1.0,19835,6,5.0


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [41]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,19835,1,3.5,3.5
1,1.0,19835,2,3.5,3.5
2,1.0,19835,3,3.5,3.5
3,1.0,19835,5,2.5,2.5
4,1.0,19835,6,5.0,5.0


In [42]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head(10)

,sum_similarityIndex,sum_weightedRating
movieId,,
1,31.148048,125.671933
2,18.849908,53.742714
3,29.670874,95.491043
4,7.322126,17.205788
5,19.137784,53.463520
6,20.406438,73.879324
7,18.046982,56.204193
8,6.034364,16.215848
9,5.113678,10.485126


### Empty dataset

In [43]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()

In [44]:
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
1,4.034665,1
2,2.851086,2
3,3.218343,3
4,2.349835,4
5,2.793611,5
6,3.620393,6
7,3.114326,7
8,2.687251,8
9,2.050408,9


# Recommended movies

Now let's sort it and see the top 20 movies that the algorithm recommended!

In [45]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
159817,5.0,159817
179135,5.0,179135
65667,5.0,65667
6144,5.0,6144
64993,5.0,64993
112224,5.0,112224
86574,5.0,86574
3415,5.0,3415
70649,5.0,70649


In [46]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())]

,movieId,title,year
3320,3415,"Mirror, The (Zerkalo)",1975
6032,6144,"Traviata, La",1982
6330,6453,Man of Aran,1934
8035,8749,"Beautiful Troublemaker, The (La belle noiseuse)",1991
12542,60737,Watching the Detectives,2007
12908,64993,5 Centimeters per Second (Byôsoku 5 senchimêtoru),2007
12985,65667,Punk's Not Dead,2007
13636,70649,A City of Sadness,1989
16368,86574,Foo Fighters: Back and Forth,2011
18014,94074,Rush: Beyond the Lighted Stage,2010
